#Задаем параметры нейронов, их модель, параметры STDP и модель STDP#

In [ ]:
!pip install brian2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.4 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.datasets import mnist
#from keras.datasets import mnist
from brian2 import *
import brian2.numpy_ as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

import time
import random

In [ ]:
n_input = 13*13 #количество входных нейронов
n_output = 4 #количество выходных нейронов

"""
Задаем параметры модели возбуждающих нейронов
"""

v_rest_e = -60.*mV #потенциал покоя  - нейрон стремится к нему без внешнего возбуждения
v_reset_e = -65.*mV #потенциал сброса  - потенцаил нейрона сбрасывается до него после испускания спайка
v_thresh_e = -52.*mV #потенциал порога генерации  - уровень при потенциала, при котором нейрон генерирует свой спайк
tau_ = 100*ms #определяет скорость реакции на входные изменения, а так же насколько быстро нейрон возвращается к своему начальному состоянию после спайка
tau_ = 150*ms # Более длительная временная константа в контексте адаптации порогового потенциала означает, что нейрон медленно адаптируется к изменениям во входной активности. Это может быть полезно для предотвращения чрезмерной реактивности нейрона на кратковременные или незначительные изменения во входных сигналах.


"""
Задаем параметры STDP
"""

lr1 = 1 #коэффициент определяющий скорость обучения в модели STDP - насколько увеличивается вес, если первым активируется пресинаптический нейрон
lr2 = 1 #коэффициент определяющий скорость обучения в модели STDP - насколько уменьшается вес, если первым активируется постсинаптический нейрон
taupre = 20*ms #определяет время, в течение которого влияние пресинаптического спайка на синаптический вес сохраняется после его произошествия.
taupost = taupre # определяет время, в течение которого влияние пресинаптического спайка на синаптический вес сохраняется после его произошествия.
gmax = .05 #максимальное значение, которое может достичь синаптическая связь
dApre = .01 #отражает вклад пресинаптчиеского спайка
dApost = -dApre * taupre / taupost * 1.05
dApost *= gmax #масштабирование треккеров синаптческого веса, чтобы они изменялись соответственно максимальному значению веса
dApre *= gmax #масштабирование треккеров синаптческого веса, чтобы они изменялись соответственно максимальному значению веса


"""
Задаем модель возбуждающего нейрона
"""

neuron_e = '''
            dv/dt = (ge*(0*mV-v) + (v_rest_e-v)) / (100*ms) : volt
            dvt/dt = (v_thresh_e-vt)/(150*ms) : volt
            dge/dt = -ge / (5*ms) : 1
            '''

"""
Задаем адаптивный порог
"""

reset_e = '''
        v = v_reset_e
        vt += 3*mV
        '''

"""
Задаем модель STDP
"""

stdp='''w : 1
    lr1 : 1
    lr2 : 1
    dApre/dt = -Apre / taupre : 1 (event-driven)
    dApost/dt = -Apost / taupost : 1 (event-driven)'''
pre='''ge += w
    Apre += dApre
    w = clip(w + lr1*Apost, 0, gmax)'''
post='''Apost += dApost
    w = clip(w + lr2*Apre, 0, gmax)'''

#Создаем класс модели#

В данной модели используется концепция подгрупп нейронов, где каждая подгруппа возбуждающих нейронов ассоциирована с определенным классом. Это означает, что во время обучения активность нейронов модулируется в зависимости от того, соответствует ли подгруппа текущему целевому классу обрабатываемого образца

In [ ]:
class Model():

  def __init__(self, debug = False):

    app = {}

    app['PG'] = PoissonGroup(n_input, rates=np.zeros(n_input)*Hz, name='PG') #создаем входные Пуассоновские нейроны, пропорциональные яркости изображения

    app['out_ne'] = NeuronGroup(n_output, neuron_e, threshold='v>vt', refractory=5*ms, reset=reset_e, method='euler', name='out_ne') #создадим выходные нейроны

    """
    Соеденяем Пуассоновские нейроны с выходными нейронами
    """

    for cl in range(n_output):

            app[f'S1_{cl}'] = Synapses(app['PG'], app['out_neu'][cl:cl+1], stdp, on_pre=pre, on_post=post, method='euler', name=f'S1_{cl}') #создаем синапсы между входными нейронами и выходными нейронами
            app[f'S1_{cl}'].connect() #соеденяем нейроны синапсами
            app[f'S1_{cl}'].w = 'rand()*gmax' #инициализируем веса
            app[f'S1_{cl}'].lr1 = 1 #скорость обучения
            app[f'S1_{cl}'].lr2 = 1 #скорость обучения


    #создаем свою сеть в brian2
    self.net = Network(app.values())
    self.net.run(0*second)

    """
    Вспомогательная функция, чтобы удобней было обращаться к отдельным структурам модели по имени обьекта класса
    """

    def __getitem__(self, key):
        return self.net[key]

    """
    Функция для тренировки SNN

    Входные данные:

    X - тренировачная часть датасета
    targets - метки тренировочной части датасета
    epochs - количество эпох для обучения
    """

    def train(self, X, targets, epochs):

      for ep in range(epochs):
        for idx in range(len(X)):

          yt = int(targets[idx]) #ground truth

          for cl in range (len(n_output)):

            if cl == n_output.index(yt):

              self.net[f'S1_{cl}'].lr1 = 1 #STDP - устанавливаем параметры скорости обучения
              self.net[f'S1_{cl}'].lr2 = 1 #STDP - устанавливаем параметры скорости обучения

            else:

              self.net[f'S1_{cl}'].lr1 = 0 #STDP - устанавливаем параметры скорости обучения
              self.net[f'S1_{cl}'].lr2 = -1 #STDP - устанавливаем параметры скорости обучения

            self.net['PG'].rates = X[idx].ravel()*Hz #устанавливаем Пуассоновские нейроны на нужное значение
            self.net.run(0.35*second) #запускаем численный эксперимент на определенное время

            self.net['PG'].rates = np.zeros(n_input)*Hz #переводим сеть в пассивное состояние - все Пуассоновские нейроны перестают генерировать спайки
            self.net.run(0.15*second) #запускаем численный эксперимент на короткое время, чтобы стабилизировать сеть


      """
      Функция оценки точности спайковой нейронной сети

      Входные данные:

      X - тестовая часть датасета
      targets - метки тестовой части датасета
      """

      def evaluate(self, X, targets):

        for cl in range(len(n_output)):

          self.net[f'S1_{cl}'].lr1 = 0 #выключаем STDP
          self.net[f'S1_{cl}'].lr2 = 0 #выключаем STDP

        true_ans = 0 #количество правильно классифицированных картинок

        for idx in range(len(X)):

          yt = int(targets[idx]) #забираем правильные метки

          #предсказание SNN определяем по нейрону с наибольшим количеством выходных спайков

          mon = SpikeMonitor(self.net['out_ne'], record=False, name='RM')
          self.net.add(mon) #добавляем монитор

          counts = [] #массив спайков выходных нейронов

          for cl in range(len(n_output)):
            counts.append(sum(mon.count[cl,cl+1]))

          if counts.index(max(counts)) == classes.index(yt):
            true_ans+=1

          #возвразаем систему в пассивный режим для стабильности

          self.net['PG'].rates = np.zeros(n_input)*Hz
          self.net.run(0.15*second)

          self.net.remove(self.net['RM']) #освобождаем память от монитора

        return true_ans/len(X)


#Основной процесс - обучение и тестирование модели#

In [ ]:
"""
Функция, которая реализует процесс обучения и тестирования модели
Входные данные:

train_items - количество тренировочных картинок
test_items - количество тестовых картинок

"""

def train_test(x_train, x_test, y_train, y_test):

  model = Model()
  model.train(x_train, y_train, epoch = 10)

  accuracy = model.evaluate(x_test, y_test)

  print(f"Accuracy of SNN is:{accuracy}")


start = time.time()
#test1(x_train, x_test, y_train, y_test)
end = time.time()
print('SNN time:', end - start)